In [59]:
import os
import shutil
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math

In [60]:
## Creating a dataset for Pain related frames

class PainDataset(Dataset):
    
    def __init__(self):
        # Data Loading from csv files to numpy and torch tensors
        xy = np.loadtxt('C:/Users/Nischal/Desktop/BioVid_split/pain/Pain_features_and_labels_071309_w_21_pain.csv', delimiter=",",dtype=np.float32,skiprows=1)
        self.x=torch.from_numpy(xy[:7921,:-1])
        self.y=torch.from_numpy(xy[:,[-1]])
        self.n_samples = 7920 ## Adjusted total number of the frames to match as a multiple of features*sequence
        
    def __getitem__(self, index):
        # dataset[0]
        return self.x[index], self.y[index]
    
    def __len__(self):
        # Len(dataset)
        return self.n_samples
        

In [61]:
# Test code to check length of dataset

# dataset = PainDataset()
# print(len(PainDataset()))
# dataloader = DataLoader(dataset=dataset, batch_size=10, shuffle= False)

# dataiter = iter(dataloader)
# data = dataiter.next()
# features, labels =data
# print(len(features[0]))

In [77]:
## Main part of code, Building architecture and training for the Pain labels

# Device configuration for CPU or GPU
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
device = torch.device('cuda')

# Hyper-parameters 
# input_size = 18 AU (Both presence(1) and intensities(17) )
num_classes = 2 ## (Binary classification as pain(0) or no pain(1) )
num_epochs = 5
batch_size = 10
learning_rate = 0.0001

input_size = 18
sequence_length = 10
hidden_size = 128
num_layers = 2


## PAIN DATASET

dataset = PainDataset()

train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle= False)
test_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle= False)


# Fully connected neural network with one hidden layer
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        # self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)
        
        # or:
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # Can add sigmoid here itself ################################################## 
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        # Check list sizes here to match required lengths
        # x: (n, 28, 28), h0: (2, n, 128)
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0,c0))  
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
        
        out = self.fc(out)
        # out: (n, 2)
        return out

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer setting
# criterion = nn.CrossEntropyLoss() ## Useful for later stage where we might have to use pain levels
criterion = nn.BCELoss()
m = nn.Sigmoid()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  


In [78]:
## Pain label training for the LSTM model

# Train the model
n_total_steps = len(train_loader)
output_list=[]
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        features = features.reshape(-1, sequence_length, input_size).to(device)
#         labels=torch.ones([2], dtype=torch.float32)
        a=labels.numpy()
        a=np.round(a.mean())
        a=[a,a]
        labels=torch.tensor(a,dtype= torch.float32)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(features)
        output_list.append(outputs[0][0])
        loss = criterion(m(outputs), labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
#             print(len(output_list))
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/5], Step [100/792], Loss: 0.0669
Epoch [1/5], Step [200/792], Loss: 0.0088
Epoch [1/5], Step [300/792], Loss: 0.0058
Epoch [1/5], Step [400/792], Loss: 0.0041
Epoch [1/5], Step [500/792], Loss: 0.0032
Epoch [1/5], Step [600/792], Loss: 0.0025
Epoch [1/5], Step [700/792], Loss: 0.0020
Epoch [2/5], Step [100/792], Loss: 0.0014
Epoch [2/5], Step [200/792], Loss: 0.0012
Epoch [2/5], Step [300/792], Loss: 0.0011
Epoch [2/5], Step [400/792], Loss: 0.0009
Epoch [2/5], Step [500/792], Loss: 0.0008
Epoch [2/5], Step [600/792], Loss: 0.0007
Epoch [2/5], Step [700/792], Loss: 0.0006
Epoch [3/5], Step [100/792], Loss: 0.0005
Epoch [3/5], Step [200/792], Loss: 0.0005
Epoch [3/5], Step [300/792], Loss: 0.0004
Epoch [3/5], Step [400/792], Loss: 0.0004
Epoch [3/5], Step [500/792], Loss: 0.0004
Epoch [3/5], Step [600/792], Loss: 0.0003
Epoch [3/5], Step [700/792], Loss: 0.0003
Epoch [4/5], Step [100/792], Loss: 0.0003
Epoch [4/5], Step [200/792], Loss: 0.0002
Epoch [4/5], Step [300/792], Loss:

In [79]:
# SAVING THE MODEL
PATH =r'C:\Users\Nischal\Desktop\LSTM model\saved_model\pain_model.pth'
torch.save(model.state_dict(), PATH)



# torch.save(model.state_dict(), 'path')
# # weights/xyz.pt
# model.load_state_dict(torch.load('path', map_location=torch.device('cpu')))

In [80]:
model.load_state_dict(torch.load(PATH, map_location=torch.device('cuda')))

<All keys matched successfully>

In [81]:
class NoPainDataset(Dataset):
    
    def __init__(self):
        # Data Loading
        xy = np.loadtxt('C:/Users/Nischal/Desktop/BioVid_split/no_pain/Pain_features_and_labels_071309_w_21_no_pain.csv', delimiter=",",dtype=np.float32,skiprows=1)
        self.x=torch.from_numpy(xy[:7921,:-1])
        self.y=torch.from_numpy(xy[:,[-1]])
        self.n_samples = 7920
        
    def __getitem__(self, index):
        # dataset[0]
        return self.x[index], self.y[index]
    
    def __len__(self):
        # Len(dataset)
        return self.n_samples

In [82]:
dataset = NoPainDataset()

train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle= False)
test_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle= False)

## No Pain label training for the LSTM model
# model.train()
# Train the model
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        features = features.reshape(-1, sequence_length, input_size).to(device)
#         labels=torch.ones([2], dtype=torch.float32)
        a=labels.numpy()
        a=np.round(a.mean())
        a=[a,a]
        labels=torch.tensor(a,dtype= torch.float32)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(features)
#         print(outputs)
        loss = criterion(m(outputs), labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/5], Step [100/792], Loss: 0.4674
Epoch [1/5], Step [200/792], Loss: 0.0437
Epoch [1/5], Step [300/792], Loss: 0.0137
Epoch [1/5], Step [400/792], Loss: 0.0078
Epoch [1/5], Step [500/792], Loss: 0.0055
Epoch [1/5], Step [600/792], Loss: 0.0042
Epoch [1/5], Step [700/792], Loss: 0.0035
Epoch [2/5], Step [100/792], Loss: 0.0030
Epoch [2/5], Step [200/792], Loss: 0.0026
Epoch [2/5], Step [300/792], Loss: 0.0023
Epoch [2/5], Step [400/792], Loss: 0.0018
Epoch [2/5], Step [500/792], Loss: 0.0017
Epoch [2/5], Step [600/792], Loss: 0.0015
Epoch [2/5], Step [700/792], Loss: 0.0014
Epoch [3/5], Step [100/792], Loss: 0.0014
Epoch [3/5], Step [200/792], Loss: 0.0013
Epoch [3/5], Step [300/792], Loss: 0.0012
Epoch [3/5], Step [400/792], Loss: 0.0010
Epoch [3/5], Step [500/792], Loss: 0.0009
Epoch [3/5], Step [600/792], Loss: 0.0009
Epoch [3/5], Step [700/792], Loss: 0.0008
Epoch [4/5], Step [100/792], Loss: 0.0008
Epoch [4/5], Step [200/792], Loss: 0.0008
Epoch [4/5], Step [300/792], Loss:

In [83]:
## Testing the model for pain labels

model.eval()

dataset = PainDataset()
test_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle= False)

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for features, labels in test_loader:
        features = features.reshape(-1, sequence_length, input_size).to(device)
#         print('Feature',features[0])
        a=labels.numpy()
        a=np.round(a.mean())
        a=[a,a]
        labels=torch.tensor(a,dtype= torch.float32).to(device)
#         labels=torch.ones([2], dtype=torch.float32)
        outputs = model(features)
        outputs=m(outputs)
#         print('labels',labels)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
#         print(predicted)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on test data: {acc} %')

Accuracy of the network on test data: 0.0 %


In [84]:
## Testing the model for no pain labels

dataset = NoPainDataset()

train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle= False)
test_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle= False)


with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for features, labels in test_loader:
        features = features.reshape(-1, sequence_length, input_size).to(device)
        a=labels.numpy()
        a=np.round(a.mean())
        a=[a,a]
        labels=torch.tensor(a,dtype= torch.float32).to(device)
#         print(labels)
        outputs = model(features)
        outputs=m(outputs)
#         print(outputs)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
#         print(predicted)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on test data: {acc} %')

Accuracy of the network on test data: 100.0 %


In [53]:
a=[0., 0.]
test=torch.tensor(a, device='cuda:0')
torch.max(test, 1)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [85]:
# Test code to check length of dataset

dataset = PainDataset()
print(len(PainDataset()))
dataloader = DataLoader(dataset=dataset, batch_size=10, shuffle= False)

dataiter = iter(dataloader)
data = dataiter.next()
features, labels =data
print(len(features))

7920
10


In [86]:
features = features.reshape(-1, sequence_length, input_size).to(device)
print(features.shape)
ou = model(features)
ou=m(ou)
print(ou)

torch.Size([1, 10, 18])
tensor([[0.0004, 0.0004]], device='cuda:0', grad_fn=<SigmoidBackward>)
